In [1]:
# from google.colab import drive
# drive.mount('/content/Drive')

Mounted at /content/Drive


In [1]:
!pip install torch
!pip install transformers
!pip install tqdm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 222.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 190.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 238.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 225.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 209.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 194.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import json
import random
import warnings
import torch
import time
import argparse
import json
import os
import time
import copy
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional


from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaModel

/environment/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# List of relation types
keys = ['no_relation', 'per:title', 'org:top_members/employees',
        'org:country_of_headquarters', 'per:parents', 'per:age',
        'per:countries_of_residence', 'per:children', 'org:alternate_names',
        'per:charges', 'per:cities_of_residence', 'per:origin', 'org:founded_by',
        'per:employee_of', 'per:siblings', 'per:alternate_names', 'org:website',
        'per:religion', 'per:stateorprovince_of_death', 'org:parents',
        'org:subsidiaries', 'per:other_family', 'per:stateorprovinces_of_residence',
        'org:members', 'per:cause_of_death', 'org:member_of',
        'org:number_of_employees/members', 'per:country_of_birth',
        'org:shareholders', 'org:stateorprovince_of_headquarters',
        'per:city_of_death', 'per:date_of_birth', 'per:spouse',
        'org:city_of_headquarters', 'per:date_of_death', 'per:schools_attended',
        'org:political/religious_affiliation', 'per:country_of_death',
        'org:founded', 'per:stateorprovince_of_birth', 'per:city_of_birth',
        'org:dissolved']

# Assigning indices to the list elements and storing them in a dictionary
rel2id = {key: idx for idx, key in enumerate(keys)}
id2rel = {idx: key for key, idx in rel2id.items()}

# Printing the dictionaries
print(rel2id)
print(id2rel)

{'no_relation': 0, 'per:title': 1, 'org:top_members/employees': 2, 'org:country_of_headquarters': 3, 'per:parents': 4, 'per:age': 5, 'per:countries_of_residence': 6, 'per:children': 7, 'org:alternate_names': 8, 'per:charges': 9, 'per:cities_of_residence': 10, 'per:origin': 11, 'org:founded_by': 12, 'per:employee_of': 13, 'per:siblings': 14, 'per:alternate_names': 15, 'org:website': 16, 'per:religion': 17, 'per:stateorprovince_of_death': 18, 'org:parents': 19, 'org:subsidiaries': 20, 'per:other_family': 21, 'per:stateorprovinces_of_residence': 22, 'org:members': 23, 'per:cause_of_death': 24, 'org:member_of': 25, 'org:number_of_employees/members': 26, 'per:country_of_birth': 27, 'org:shareholders': 28, 'org:stateorprovince_of_headquarters': 29, 'per:city_of_death': 30, 'per:date_of_birth': 31, 'per:spouse': 32, 'org:city_of_headquarters': 33, 'per:date_of_death': 34, 'per:schools_attended': 35, 'org:political/religious_affiliation': 36, 'per:country_of_death': 37, 'org:founded': 38, 'per

In [4]:
# tell pytorch to use the gpu if available
if torch.cuda.is_available():

    DEVICE = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not
else:
    print('No GPU available, using the CPU instead.')
    DEVICE = torch.device("cpu")

USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    print("using GPU")
else:
    print("using CPU")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 4090
using GPU


In [5]:
#RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)
print('RoBERTa tokenizer loaded')

RoBERTa tokenizer loaded


In [6]:
def setup_seed(seed):
    # Sets the seed for generating random numbers for the CPU.

    # Sets the seed for generating random numbers for all GPUs.
    torch.cuda.manual_seed_all(seed)

    # Sets the seed for generating random numbers with NumPy.
    np.random.seed(seed)

    # Sets the seed for the built-in Python random module.
    random.seed(seed)

setup_seed(44)

In [7]:
def load_tacred_dataset(file_path):
    """
    Load the TACRED dataset from a JSON file.

    Args:
    file_path (str): The path to the JSON file containing the dataset.

    Returns:
    dict: The loaded dataset.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [8]:
def prepare_data(data):
    # Get the number of instances in the data
    n = len(data)

    # Initialize an empty list to store processed data
    info = []

    # Iterate through each instance in the data
    for i in range(n):
        # Initialize an empty dictionary for storing processed information of a single data instance
        single_data = {}

        # Extract start and end indices of subject and object entities
        ss = data[i]["subj_start"]
        se = data[i]["subj_end"]
        os = data[i]["obj_start"]
        oe = data[i]["obj_end"]

        # Extract subject and object tokens based on their start and end indices
        subj = data[i]['token'][ss: se+1]
        obj = data[i]['token'][os: oe+1]

        temp = data[i]['token'].copy()

        temp[ss: se+1] = ['[MASK]']
        temp[os: oe+1] = ['[MASK]']

        # Convert subject and object tokens into strings
        ent1 = ' '.join(subj)
        ent2 = ' '.join(obj)

        # Extract relation label
        rel = data[i]["relation"]

        # Concatenate all tokens to form the original sentence
        text = " ".join(temp)

        # Store processed information in the dictionary
        single_data['rel'] = rel
        single_data['ent1'] = ent1
        single_data['ent2'] = ent2
        single_data['text'] = text

        # Append processed information of a single data instance to the list
        info.append(single_data)

    # Return the processed data
    return info

In [9]:
def process_relation_extraction_data(info, max_length=64):
    # Initialize a dictionary to store processed data
    data = {}
    data['label'] = []  # List to store relation labels
    data['mask'] = []   # List to store attention masks
    data['text'] = []   # List to store tokenized and padded texts

    # Iterate through each instance in the provided 'info' data
    for line in info:
        # Check if the relation label is present in the 'rel2id' dictionary
        if line['rel'] not in rel2id:
            # If the relation label is not found, assign label 0 (for unknown relation)
            data['label'].append(0)
        else:
            # If the relation label is found, assign its corresponding index
            data['label'].append(rel2id[line['rel']])

        # Concatenate subject, object, and text to form a single sentence
        sent = line['ent1'] +'[SEP]'+line['ent2'] + '[SEP]' + line['text']

        # Tokenize the concatenated sentence and add special tokens
        indexed_tokens = tokenizer.encode(sent, add_special_tokens=True)

        # Determine the available length of the tokenized sentence
        avai_len = len(indexed_tokens)

        # Pad the tokenized sentence with 0s to match the maximum length
        while len(indexed_tokens) < max_length:
            indexed_tokens.append(0)

        # Trim the tokenized sentence to the maximum length
        indexed_tokens = indexed_tokens[:max_length]

        # Convert the tokenized sentence to a PyTorch tensor
        indexed_tokens = torch.tensor(indexed_tokens).long().unsqueeze(0)  # (1, L)

        # Create an attention mask for the tokenized sentence
        att_mask = torch.zeros(indexed_tokens.size()).long()  # (1, L)
        att_mask[0, :avai_len] = 1

        # Append the tokenized and padded sentence, and its attention mask to the data dictionary
        data['text'].append(indexed_tokens)
        data['mask'].append(att_mask)

    return data

In [10]:
def convert_data_to_tensors(data):
    # Extract text, mask, and label from the data
    text = data['text']
    mask = data['mask']
    label = data['label']

    # Convert text and mask tensors to numpy arrays
    text = [t.numpy() for t in text]
    mask = [t.numpy() for t in mask]

    # Convert numpy arrays to PyTorch tensors
    text = torch.tensor(text)
    mask = torch.tensor(mask)
    label = torch.tensor(label)

    return text, mask, label

In [12]:
class RoBERTa_Classifier(nn.Module):
    def __init__(self, label_num):
        super().__init__()
        # Initialize the RoBERTa encoder from pre-trained weights
        self.encoder = RobertaModel.from_pretrained("roberta-base")
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(0.1, inplace=False)
        # Fully connected layer for classification
        self.fc = nn.Linear(768, label_num)  # 768 is the hidden size of RoBERTa
        # Cross-entropy loss criterion
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x, attention_mask, label=None):
        # Pass the input through the RoBERTa encoder
        x = self.encoder(x, attention_mask=attention_mask)[0]  # Output is tuple (last_hidden_state, pooler_output), we take the last_hidden_state
        # Take only the first token's output (CLS token)
        x = x[:, 0, :]
        # Apply dropout
        x = self.dropout(x)
        # Pass through the fully connected layer
        x = self.fc(x)
        # If label is not provided, return logits only
        if label is None:
            return None, x
        else:
            # Calculate the cross-entropy loss and return both loss and logits
            return self.criterion(x, label), x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")        
labels_num=len(rel2id)
model = RoBERTa_Classifier(labels_num)
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RoBERTa_Classifier(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [13]:
def train(net, train_dataset, dev_dataset, num_epochs, learning_rate, batch_size):

    print('Training...')

    # Set the network to training mode
    net.train()

    # Define the optimizer
    optimizer = optim.SGD(net.parameters(), lr=learning_rate)

    # Create a data loader for training data
    train_iter = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)

    for epoch in range(num_epochs):
        correct = 0
        total = 0
        iter = 0
        all_pred = []
        all_true = []

        # Initialize tqdm to show progress bar
        progress_bar = tqdm(train_iter, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch')

        for text, mask, y in progress_bar:
            iter += 1
            optimizer.zero_grad()

            # If the batch size is not equal to the specified batch size, break the loop
            if text.size(0) != batch_size:
                break

            # Reshape text and mask tensors
            text = text.reshape(batch_size, -1)
            mask = mask.reshape(batch_size, -1)

            # Move tensors to GPU if available
            if USE_CUDA:
                text = text.cuda()
                mask = mask.cuda()
                y = y.cuda()

            # Forward pass
            loss, logits = net(text, mask, y)

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Compute accuracy
            _, predicted = torch.max(logits.data, 1)
            total += text.size(0)
            correct += predicted.data.eq(y.data).cpu().sum()

            # Collect predictions and true labels
            all_pred.extend(predicted.cpu().numpy())
            all_true.extend(y.cpu().numpy())

            # Update progress bar
            progress_bar.set_postfix({'loss': loss.item(), 'accuracy': correct.item() / total})

        # After the end of each epoch, compute metrics
        accuracy = correct.cpu().numpy().tolist()/total
        loss = loss.detach().cpu()

        # Compute F1 scores
        macro_f1 = f1_score(all_true, all_pred, average='macro')
        micro_f1 = f1_score(all_true, all_pred, average='micro')
        weighted_f1 = f1_score(all_true, all_pred, average='weighted')

        # Print metrics
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Loss: {loss.mean().numpy().tolist()}, Accuracy: {accuracy}")
        print(f"Macro F1: {macro_f1}, Micro F1: {micro_f1}, Weighted F1: {weighted_f1}")

        # print()

        # print("Validation...")

        # dev_acc, dev_macro_f1, dev_micro_f1, dev_weighted_f1, _, _ = eval(net, dev_dataset, batch_size)


In [14]:

def eval(net, dataset, batch_size):
    # Set the network to evaluation mode
    net.eval()

    # Create an iterator for the evaluation dataset
    eval_iter = DataLoader(dataset, batch_size, shuffle=False)

    # Lists to store predictions and true labels
    all_pred = []
    all_true = []

    # Lists to store evaluation metrics
    acc_list = []
    macro_f1_list = []
    micro_f1_list = []
    weighted_f1_list = []
    precision_list = []  # List for precision scores
    recall_list = []     # List for recall scores

    with torch.no_grad():
        correct = 0  # Counter for correctly classified samples
        total = 0    # Counter for total samples
        # Progress bar for visualization during evaluation
        progress_bar = tqdm(eval_iter, desc='Evaluation', unit='batch')

        for text, mask, y in progress_bar:
            if text.size(0) != batch_size:
                break

            text = text.reshape(batch_size, -1)
            mask = mask.reshape(batch_size, -1)

            if USE_CUDA:
                text, mask, y = text.cuda(), mask.cuda(), y.cuda()

            outputs = net(text, mask)
            loss, logits = outputs if isinstance(outputs, tuple) else (None, outputs)

            _, predicted = torch.max(logits, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

            all_pred.extend(predicted.cpu().numpy())
            all_true.extend(y.cpu().numpy())

        # Calculate overall accuracy and F1 scores
        acc = correct / total
        macro_f1 = f1_score(all_true, all_pred, average='macro')
        micro_f1 = f1_score(all_true, all_pred, average='micro')
        weighted_f1 = f1_score(all_true, all_pred, average='weighted')

        # Calculate precision and recall
        precision_macro = precision_score(all_true, all_pred, average='macro')
        recall_macro = recall_score(all_true, all_pred, average='macro')
        precision_micro = precision_score(all_true, all_pred, average='micro')
        recall_micro = recall_score(all_true, all_pred, average='micro')
        precision_weighted = precision_score(all_true, all_pred, average='weighted')
        recall_weighted = recall_score(all_true, all_pred, average='weighted')

        # Append metrics to respective lists
        acc_list.append(acc)
        macro_f1_list.append(macro_f1)
        micro_f1_list.append(micro_f1)
        weighted_f1_list.append(weighted_f1)
        precision_list.append((precision_macro, precision_micro, precision_weighted))
        recall_list.append((recall_macro, recall_micro, recall_weighted))

        # Print evaluation results
        print(f"Eval Result: Acc: {acc:.4f}, Macro F1: {macro_f1:.4f}, Micro F1: {micro_f1:.4f}, Weighted F1: {weighted_f1:.4f}")
        print(f"Precision (Macro, Micro, Weighted): {precision_macro:.4f}, {precision_micro:.4f}, {precision_weighted:.4f}")
        print(f"Recall (Macro, Micro, Weighted): {recall_macro:.4f}, {recall_micro:.4f}, {recall_weighted:.4f}")

        # Return evaluation metrics
        return acc_list, macro_f1_list, micro_f1_list, weighted_f1_list, precision_list, recall_list


In [15]:
# Load data from JSON file
train_data = load_tacred_dataset('/home/featurize/data/TACRED/train.json')
dev_data = load_tacred_dataset('/home/featurize/data/TACRED/dev.json')
test_data = load_tacred_dataset('/home/featurize/data/TACRED/test.json')

In [16]:
train_info = prepare_data(train_data)
dev_info = prepare_data(dev_data)
test_info = prepare_data(test_data)

print(len(train_info))
print(len(dev_info))
print(len(test_info))
print(train_info[1])

68124
22631
15509
{'rel': 'no_relation', 'ent1': 'Forsberg', 'ent2': 'John D.', 'text': "In 1983 , a year after the rally , [MASK] received the so-called `` genius award '' from the [MASK] and Catherine T. MacArthur Foundation ."}


In [17]:
train_data = process_relation_extraction_data(train_info, 128)
dev_data = process_relation_extraction_data(dev_info, 128)
test_data = process_relation_extraction_data(test_info, 128)


print(len(train_data['label']))
print(len(dev_data['label']))
print(len(test_data['label']))

68124
22631
15509


In [18]:
# Preprocess train data
train_text, train_mask, train_label = convert_data_to_tensors(train_data)
print("--train data--")
print(train_text.shape)
print(train_mask.shape)
print(train_label.shape)

# Preprocess dev data
dev_text, dev_mask, dev_label = convert_data_to_tensors(dev_data)
print("--eval data--")
print(dev_text.shape)
print(dev_mask.shape)
print(dev_label.shape)

# Preprocess test data
test_text, test_mask, test_label = convert_data_to_tensors(test_data)
print("--test data--")
print(test_text.shape)
print(test_mask.shape)
print(test_label.shape)

/tmp/ipykernel_20642/3242763015.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  text = torch.tensor(text)


--train data--
torch.Size([68124, 1, 128])
torch.Size([68124, 1, 128])
torch.Size([68124])
--eval data--
torch.Size([22631, 1, 128])
torch.Size([22631, 1, 128])
torch.Size([22631])
--test data--
torch.Size([15509, 1, 128])
torch.Size([15509, 1, 128])
torch.Size([15509])


In [19]:
train_dataset = torch.utils.data.TensorDataset(train_text,train_mask,train_label)
dev_dataset = torch.utils.data.TensorDataset(dev_text,dev_mask,dev_label)

In [20]:
train(model, train_dataset, dev_dataset, 15, 0.002, 16)

Training...


Epoch 1/15: 100%|█████████▉| 4257/4258 [02:53<00:00, 24.59batch/s, loss=0.635, accuracy=0.831] 


Epoch 1/15
Loss: 0.6349697113037109, Accuracy: 0.8310870331219169
Macro F1: 0.1148647140638592, Micro F1: 0.8310870331219169, Weighted F1: 0.7882030084862661


Epoch 2/15: 100%|█████████▉| 4257/4258 [02:52<00:00, 24.62batch/s, loss=0.0896, accuracy=0.866]


Epoch 2/15
Loss: 0.08964329212903976, Accuracy: 0.866014799154334
Macro F1: 0.3221312394994451, Micro F1: 0.866014799154334, Weighted F1: 0.846426814901049


...

Epoch 13/15
Loss: 0.024845732375979424, Accuracy: 0.9264299976509279
Macro F1: 0.6950956727078037, Micro F1: 0.9264299976509279, Weighted F1: 0.9239417434475099


Epoch 14/15: 100%|█████████▉| 4257/4258 [02:52<00:00, 24.72batch/s, loss=0.306, accuracy=0.93]   


Epoch 14/15
Loss: 0.3058245778083801, Accuracy: 0.9304087385482734
Macro F1: 0.7225880768179801, Micro F1: 0.9304087385482734, Weighted F1: 0.9284947718525174


Epoch 15/15: 100%|█████████▉| 4257/4258 [02:55<00:00, 24.32batch/s, loss=0.101, accuracy=0.934]  


Epoch 15/15
Loss: 0.10093909502029419, Accuracy: 0.9337121212121212
Macro F1: 0.729940844101015, Micro F1: 0.9337121212121212, Weighted F1: 0.9318697662441596


In [21]:
torch.save(model, 'roberta_best_model_withmask.pth')
torch.save(model.state_dict(), 'roberta_best_model_withmask2.pth')
print("entire Model saved successfully.")

entire Model saved successfully.


In [22]:
test_dataset = torch.utils.data.TensorDataset(test_text, test_mask, test_label)
acc_list, macro_f1_list, micro_f1_list, weighted_f1_list, precision_list, recall_list= eval(model, test_dataset, 32)

Evaluation: 100%|█████████▉| 484/485 [00:12<00:00, 39.07batch/s]
/environment/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Eval Result: Acc: 0.8768, Macro F1: 0.5516, Micro F1: 0.8768, Weighted F1: 0.8744
Precision (Macro, Micro, Weighted): 0.5683, 0.8768, 0.8746
Recall (Macro, Micro, Weighted): 0.5573, 0.8768, 0.8768


/environment/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
